## E+ in zeppy
Here we try to run E+ thru zeppy

Lets start by using eppy to do this
(make sure you have eppy installed before you do this.)

In [1]:
# setting the path name so that the tutorial works from this directory
# if you install zeppy with `pip install zeppy` you will not need to do this
import sys
# pathnameto_zeppy = 'c:/eppy'
pathnameto_zeppy = '../../'
sys.path.append(pathnameto_zeppy)

In [2]:
import os
import eppy
from zeppy import ppipes
wfile = "/Applications/EnergyPlus-9-1-0/WeatherData/USA_CO_Golden-NREL.724666_TMY3.epw"

In [3]:
fnames = [
    # "./eplus_files/Minimal.idf",
        "../../eplus_files/UnitHeaterGasElec.idf",
        "../../eplus_files/ZoneWSHP_wDOAS.idf",
        "../../eplus_files/ZoneWSHP_wDOAS_1.idf",
        ]
wfile = "/Applications/EnergyPlus-9-1-0/WeatherData/USA_CO_Golden-NREL.724666_TMY3.epw"
idfs = [eppy.openidf(fname, epw=wfile) for fname in fnames]

Let us write a couple of helper functions

In [4]:
def idf_multirun(idf_kwargs):
    import eppy
    # for some reason idf.run() does not work
    # so I am using runIDFs to run one idf at a time
    idf = idf_kwargs['args']
    options = idf_kwargs['kwargs']
    eppy.runner.run_functions.runIDFs( [(idf, options)] ) 


def make_options(idf):
    idfversion = idf.idfobjects['version'][0].Version_Identifier.split('.')
    idfversion.extend([0] * (3 - len(idfversion)))
    idfversionstr = '-'.join([str(item) for item in idfversion])
    fname = idf.idfname
    options = {
        'ep_version':idfversionstr,
        'output_prefix':os.path.basename(fname).split()[0],
        'output_suffix':'C',
        'output_directory':os.path.dirname(fname),
        }
    return options


Now let us run the E+ files

In [5]:
waitlist = [[{'args':idf, 'kwargs':make_options(idf)}] for idf in idfs]
func = idf_multirun
result = ppipes.ipc_parallelpipe(func, waitlist, nworkers=None, verbose=True)
print(result)

started worker 0
started worker 1
started worker 2
starting sink
number of calculations = 3
sleeptime=0.1
started ventilator
running item: 0, in worker: 0

/Applications/EnergyPlus-9-1-0/energyplus --weather /Applications/EnergyPlus-9-1-0/WeatherData/USA_CO_Golden-NREL.724666_TMY3.epw --output-directory /Users/santoshphilip/Documents/coolshadow/temp/newrepo/zeppy/eplus_files --output-prefix UnitHeaterGasElec.idf --output-suffix C /Users/santoshphilip/Documents/coolshadow/temp/newrepo/zeppy/docs/tutorial_docs/multi_runs/idf_0/in.idf

sleeptime=0.1
sleeptime=0.1
running item: 2, in worker: 1

/Applications/EnergyPlus-9-1-0/energyplus --weather /Applications/EnergyPlus-9-1-0/WeatherData/USA_CO_Golden-NREL.724666_TMY3.epw --output-directory /Users/santoshphilip/Documents/coolshadow/temp/newrepo/zeppy/eplus_files --output-prefix ZoneWSHP_wDOAS_1.idf --output-suffix C /Users/santoshphilip/Documents/coolshadow/temp/newrepo/zeppy/docs/tutorial_docs/multi_runs/idf_0/in.idf

sent result of item:

### Ha! all three files run. 
The results are `None`, since we did not write code to exract and return any outputs from the E+ outputs

Let us try running on a single worker and see how long it takes:

In [6]:
result = ppipes.ipc_parallelpipe(func, waitlist, nworkers=1, verbose=True)
print(result)

started worker 0
starting sink
number of calculations = 3
started ventilator
sleeptime=0.1
running item: 0, in worker: 0

/Applications/EnergyPlus-9-1-0/energyplus --weather /Applications/EnergyPlus-9-1-0/WeatherData/USA_CO_Golden-NREL.724666_TMY3.epw --output-directory /Users/santoshphilip/Documents/coolshadow/temp/newrepo/zeppy/eplus_files --output-prefix UnitHeaterGasElec.idf --output-suffix C /Users/santoshphilip/Documents/coolshadow/temp/newrepo/zeppy/docs/tutorial_docs/multi_runs/idf_0/in.idf

sleeptime=0.1
sleeptime=0.1
sent result of item: 0, in worker: 0 to sink
running item: 1, in worker: 0

/Applications/EnergyPlus-9-1-0/energyplus --weather /Applications/EnergyPlus-9-1-0/WeatherData/USA_CO_Golden-NREL.724666_TMY3.epw --output-directory /Users/santoshphilip/Documents/coolshadow/temp/newrepo/zeppy/eplus_files --output-prefix ZoneWSHP_wDOAS.idf --output-suffix C /Users/santoshphilip/Documents/coolshadow/temp/newrepo/zeppy/docs/tutorial_docs/multi_runs/idf_0/in.idf

sent result

OK. Took a bit longer running on just one worker

### Multi core machines

- some superficial testing has shown that on a multicore machine, each run is on a different core.
- you can force the number of cores running by setting a value for `nworkers`
- if you have more runs than cores. Say you have 4 cores and 50 runs, you can set `nworker=4`. The runs would be spread evenly on all the cores. About 12 or 13 runs on each core. Each core will execute the runs sequentially.
- If you have 4 cores and assign 10 workers, it will still run. We have not checked ot see how it does it.